In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys;
sys.path.append(r'C:\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy
import concurrent.futures

import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
import statsmodels.api as sm

import APIs.QuickStats as qs

import Utilities.SnD as us
import Utilities.Weather as uw
import Utilities.Modeling as um

import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy

In [4]:
scope = cy.Define_Scope()

raw_data = cy.Get_Data_All_Parallel(scope)
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units='in', temp_units='F',ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)
model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [19]:
scenarios_EXT_dict = {}
for p in range(1985,2022):
    for t in range(1985,2022):
        scenarios_EXT_dict[str(p)+'_'+str(t)]={GV.WV_PREC:GV.EXT_ANALOG+'_'+str(p),GV.WV_SDD_30:GV.EXT_ANALOG+'_'+str(t)}

print(len(scenarios_EXT_dict))
# print(scenarios_EXT_dict)

1369


In [ ]:
%%time
results = {}
i=0
for sce_name, sce_dict in scenarios_EXT_dict.items():
    i+=1; print(i)
    pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F', ext_mode = sce_dict)
    pred_df = cy.Build_Pred_DF(raw_data,milestones,pred_DF_instr,GV.CUR_YEAR, dt(2022,7,5))
    yields = model.predict(pred_df[model.params.index]).values

    results[sce_name]=yields[0]

[print(k,'->',v) for k,v in results.items()]
print(results)
print('All Done')

In [23]:
results['2013_2012']

143.97644417218277

In [21]:
min(results.values())

143.97644417218277

In [22]:
min(results, key=results.get)

'2013_2012'